<a href="https://colab.research.google.com/github/felipehigon/DatathonCajamar/blob/main/CLASE_CAJAMAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Librerias

import pandas as pd
from tqdm import tqdm


class Contador(object):
    def __init__(self, datos_contador, ID):
        self.datos_contador = datos_contador
        self.ID = ID
        self.Preprocesado()
        self.FaltaDatos = False
        self.outlier = False
        self.RMSE = 0
        self.Predicciones = {'Dia_1':0,'Dia_2':0,'Dia_3':0,.......'Semana_2':0}
        self.longitud_ST_ID=datos_contador.shape[0]
        
        # CALCULO LECTURA CONTADOR Y CONSUMO
        self.datos_contador['CONSUMO'] = self.datos_contador['DELTAINTEGER'] + self.__Convertir_Entero(self.datos_contador['DELTATHOUSANDTH'])
        self.datos_contador['LECTURA'] = self.datos_contador['READINGINTEGER'] + self.__Convertir_Entero(self.datos_contador['READINGTHOUSANDTH'])
        
    def __Convertir_Entero(self,serie):
        resultado = {}
        indices = []
        
        for indice, numero in serie.items():
            if numero < 10:
                resultado[indice] = numero/10
                indices.append(indice)
            elif numero < 100:
                resultado[indice] = numero/100
                indices.append(indice)
            elif numero >= 100:
                print('ERROR parte decimal mayor de 100')
        return pd.Series(data=resultado, index=indices)
    
    def Preprocesado(self):
        # READINGINTEGER	READINGTHOUSANDTH	DELTAINTEGER	DELTATHOUSANDTH
        # VALORES  NULOS 
        # VALORES ENTEROS INTERPOLAMOS
        # JOSEP

        self.datos_contador['READINGINTEGER'].interpolate(inplace=True)
        self.datos_contador['DELTAINTEGER'].interpolate(inplace=True)
        #self.datos_contador.fillna(0)
        #self.datos_contador['DELTAINTEGER'].fillna(0)
        
        # VALORES DECIMALES SUSTITUIMOS POR MEDIA
        #indices = self.datos_contador.notna()['READINGTHOUSANDTH']
        #media = int(self.datos_contador['READINGTHOUSANDTH'][indices].mean())
        self.datos_contador['READINGTHOUSANDTH'].fillna(0,inplace=True)
        #indices = self.datos_contador.notna()['DELTATHOUSANDTH']
        #media = int(self.datos_contador['DELTATHOUSANDTH'][indices].mean())
        self.datos_contador['DELTATHOUSANDTH'].fillna(0,inplace=True)
        
        # ELIMINAR VALORES NEGATIVOS
        indices = self.datos_contador[self.datos_contador['READINGINTEGER'] < 0].index
        self.datos_contador.drop(indices,inplace=True)
        indices = self.datos_contador[self.datos_contador['DELTAINTEGER'] < 0].index
        self.datos_contador.drop(indices,inplace=True)

        # CONDICION ULTIMOS DIAS VER GRAFICA Y FIJAR NUMERO
        
        # CONTADORES CON POCOS DATOS VEMOS SI SON NUEVOS TIENEN DATOS EN LOS ULTIMOS DIAS HACEMOS MEDIA
        # EN EL CASO DE NO TENER LOS ULTIMOS DATOS PONEMOS CERO

    def PreModelo(self, datos_contador, dias_pred):
        self.datos_contador['LECTURA_NORM']=
          (self.datos_contador['LECTURA']-self.datos_contador['LECTURA'].mean())/self.datos_contador['LECTURA'].std()
        self.datos_contador['CONSUMO_NORM']=
          (self.datos_contador['CONSUMO']-self.datos_contador['CONSUMO'].mean())/self.datos_contador['CONSUMO'].std()
        
        datos_train=self.datos_contador['CONSUMO'][:-dias_pred]
        datos_test=self.datos_contador['CONSUMO'][-dias_pred:]

        datos_train_exog=self.datos_contador['LECTURA'][:-dias_pred]
        datos_test_exog=self.datos_contador['LECTURA'][-dias_pred:]

        datos_train_norm=self.datos_contador['CONSUMO_NORM'][:-dias_pred]
        datos_test_norm=self.datos_contador['CONSUMO_NORM'][-dias_pred:]

        datos_train_exog_norm=self.datos_contador['LECTURA_NORM'][:-dias_pred]
        datos_test_exog_norm=self.datos_contador['LECTURA_NORM'][-dias_pred:]

        return datos_train, datos_test, datos_train_exog, datos_test_exog, datos_train_norm, datos_test_norm, datos_train_exog_norm, datos_test_exog_norm

        

        # PREPROCESADO PARA INTRODUCIR LOS DATOS EN EL MODELO
        # FORMATO DATOS EN FUNCIÓN DEL MODELO
        # PACO

    def modelo(self, modelos, umbrales):
      # DECISION DEL MODELO A TOMAR EN FUNCION DE LA ESTADISTICAS
      # CARGA DEL MODELO
        if self.longitud_ST_ID<umbrales[0]:
           model=modelos[0]
        elif umbrales[0] >= self.longitud_ST_ID < umbrales[1]:
           model=modelos[1]
        elif umbrales[1] >= self.longitud_ST_ID < umbrales[2]:
           model=modelos[2]
        else:
           model=modelos[3]
        
        return model
    

class CajaMar_Water(object):
    def __init__(self,Ruta_Datos):
        self.Ruta_Datos = Ruta_Datos
        self.datos = pd.DataFrame()
        self.contadores = []
        self.Cargar_Datos()
    

    def Cargar_Datos(self):
        print('CARGANDO DATOS.....')
        self.datos = pd.read_csv(self.Ruta_Datos,sep='|',parse_dates=['SAMPLETIME'],infer_datetime_format=True,encoding='utf-8')
        self.datos.sort_values(by=['ID','SAMPLETIME'],inplace=True)
         
        # GENERANDO CONTADORES
        print('GENERANDO CONTADORES.....')
        for ID in tqdm(pd.unique(self.datos['ID'])):
            # ALMACENAMOS ID DE CONTADOR Y BORRAMOS LA COLUMNA
            self.contadores.append(Contador(self.datos[self.datos['ID']==ID].drop('ID',axis=1),ID))



        # CONSUMO POR DIAS Y CONTADORES
        #print('AGRUPANDO CONSUMOS POR DIAS.....')
        #self.datos_consumo = self.datos.groupby([self.datos['ID'],self.datos['SAMPLETIME'].dt.year,self.datos['SAMPLETIME'].dt.month, self.datos['SAMPLETIME'].dt.day]).agg({'CONSUMO':sum})
        print('PROCESO DE CARGA FINALIZADO')
    
    def PartirDatos(self,dias=7):
        # Separar datos de entrenamiento y test / validacion (dias=0 no hace validacion)


    def Entrenamiento(self):
        # Entrenamiento de los modelos en self.contadores(lista) 
        # iterar lista y llamar a self.contadores[iter].modelo.train
        # Rellena la variable self.contadores.Predicciones
        

    def Calculo_RMSE(self):
        # Calculo rmse utlizando datos de test
        
    
    def GenerarResultados(self):
        # Mediante los datos predicciones generamos txt con el formato del datathon

        return fichero    